In [1]:
from vpython import*
import random
import numpy as np
import itertools

<IPython.core.display.Javascript object>

In [2]:
def box_select(box_list, product_list):
  for i in box_list:
    box_volum = i[0]*i[1]*i[2]
    box = i.copy()
    box.sort()
    product_volum_sum = 0
    for j in product_list:
       product_volum = j[0]*j[1]*j[2]
       product_volum_sum += product_volum_sum

    if box_volum < product_volum_sum:
      continue

    count = 0  #모든 상품의 모든 길이가 상자의 크기보다 작은지 확인 하기 위하여
    for k in product_list:
      k.sort()
      if k[0] <= box[0] and k[1] <= box[1] and k[2] <= box[2]:
        count += 1
    
    if len(product_list) == count:
      selected = i
      break
  
  return selected

In [3]:
def make_list(product_list):
  list = []
  for i in product_list:
    case = [[i[0],i[1],i[2]],[i[0],i[2],i[1]],[i[1],i[0],i[2]],
            [i[1],i[2],i[0]],[i[2],i[0],i[1]],[i[2],i[1],i[0]]]
    list.append(case)
  #각 상품 마다 모든 회전된 리스트릴 먼저 만들었음
  
  n=len(list)
  list_case=[]
  a = 6**n
  #6의 n승번 반복하도록 하는 거
  for i in range(a):
    list_add=[]
    for i in range(n):
      y = random.randint(0,5)
      a = list[i][y]
      list_add.append(a)
  #회전된 리스트를 가지고 6^n번 리스트를 만들었다.

    list_case.append(list_add)


  return list_case

In [4]:
def load(box,list_case):
  box_size = np.zeros((box[2],box[1],box[0]))#높이, 세로, 가로
  n = len(list_case[0]) # 하나의 케이스 길이
  product_information = []
  print(n)
  for i in list_case:
    *b, = itertools.permutations(i,n) #각 케이스의 조합을 만듬 
    for j in b: #조합의 첫번째 부터 마지막 까지 각각의 경우에 대해서 생각해보는(아마 n!개가 나올 거임)
      position = np.array([0,0,0])  #[x,y,z]
      can = True
      pos_hist=[]
      box_size = np.zeros((box[2],box[1],box[0]))
      for case in j: #각 조합의 첫번째 부터 적재가 가능한지 검사를 해 본다.
        position = np.array([0,0,0])
        
        while(True):
          
        #실질적으로 적재가 가능한지 검사하는 코드
          slice_value = box_size[position[2]:case[2]+position[2],position[1]:case[1]+position[1],position[0]:case[0]+position[0]]
          correct = np.zeros((case[2],case[1],case[0]))

          #자른 만큼의 크기와 동일하게 0으로 되어있다면 1로 채워 넣음
          if np.array_equal(slice_value,correct):
            box_size[position[2]:case[2]+position[2],position[1]:case[1]+position[1],position[0]:case[0]+position[0]] = np.ones((case[2],case[1],case[0]))
            pos_hist.append(position)
            
            break
            
         #적재가 불가능 할 경우 한칸씩 이동해 보며 확인을 해보는 경우를 생각해 보려고 함
          if position[0] < box[0]-case[0]:
            position[0] += 1

          elif position[1] < box[1]-case[1]:
            position[1] += 1
            if position[0] == box[0]-case[0]:
              position[0] = 0
         
          elif position[2] < box[2] - case[2]:
            position[2] += 1
            if position[0] == box[0]-case[0]:
              position[0] = 0

            if position[1] == box[1]-case[1]:
              position[1] = 0
            
          if position[0] >= box[0] - case[0] and position[1] >= box[1] - case[1] and position[2] >= box[2] - case[2]:
            can = False
            break 

        
        if can == False:
          break
        
        
        if len(pos_hist) == n:
          break 

      if len(pos_hist) == n:
         product_information = j
         break
    if len(pos_hist) == n:
      break
  
  return pos_hist,product_information

In [5]:
#상자가 가로 세로 높이 순서로 들어갔을 경우 우체국 박스크기 기준?
#[22, 19, 9],[27,18,15],[32,25,10],[34,25,21],[41,31,28],[48,38,34]
box_list = [[22, 19, 9],[27,18,15],[32,25,10],[34,25,21],[41,31,28],[48,38,34]]
product_list = [[15,4,15],[6,14,3],[15,4,4],[15,4,4],[15,4,15]]
#박스 리스트랑 물건 리스트 작성해야함
box_size = box_select(box_list,product_list)
#선택된 박스의 정보를 담음

list_case = make_list(product_list)

hist, product_inform = load(box_size, list_case)

5


In [6]:
print(hist)
print(box_size)
print(product_inform)

[array([0, 0, 0]), array([15,  0,  0]), array([ 0, 15,  0]), array([0, 0, 4]), array([0, 4, 4])]
[22, 19, 9]
([15, 15, 4], [6, 14, 3], [15, 4, 4], [15, 4, 4], [15, 15, 4])


In [7]:
center = vector(box_size[0]/2,box_size[2]/2,box_size[1]/2)
size = vector(box_size[0],box_size[2],box_size[1])
box_big = box(pos = center, size = size, color = color.white, opacity = 0.3)

print(hist[0])
print(product_inform[0])
center = vector(hist[0][0]+product_inform[0][0]/2,hist[0][2]+product_inform[0][2]/2,hist[0][1]+product_inform[0][1]/2)
size = vector(product_inform[0][0],product_inform[0][2],product_inform[0][1])
box_big = box(pos = center, size = size, color = color.blue )

color_list = [(1,0,0),(0,1,0),(0,0,1),(1,1,0),(1,0.5,0),(0,1,1),(1,0,1),(1,1,1)]

for i in range(len(hist)):
    s = i%8
    center = vector(hist[i][0]+product_inform[i][0]/2,hist[i][2]+product_inform[i][2]/2,hist[i][1]+product_inform[i][1]/2)
    size = vector(product_inform[i][0],product_inform[i][2],product_inform[i][1])
    box_big = box(pos = center, size = size, color = vector(color_list[s][0],color_list[s][1],color_list[s][2]))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[0 0 0]
[15, 15, 4]
